# Sequence classification with Neural Networks
## Split-window RNN model

Now we're going to try split-window RNN model. We are doing this because feeding whole sequence of your data might be impractican for a number of reasons:

* one sample of high-frequency data (like acceleration) might not even fit into GPU memory and the training will crash
* model might not be able to learn properly on long sequences
* avoids the need for padding/masking since we will have equal size windows

In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
import altair as alt

import numpy as np
import pandas as pd

import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from tmdprimer.datagen import generate_sample, Dataset, Sample

We're going to use the same network as for the per-sample RNN:

In [2]:
import tensorflow as tf

# converge faster
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        0.01,
        decay_steps=100,
        decay_rate=0.7)

def get_rnn_model():
    rnn_model = tf.keras.Sequential(
        [
            tf.keras.layers.GRU(8, return_sequences=True),
            tf.keras.layers.Dense(1, activation="sigmoid")
        ]
    )
    rnn_model.compile(
        loss="binary_crossentropy",
        optimizer=tf.keras.optimizers.RMSprop(learning_rate=lr_schedule),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )
    return rnn_model

In [3]:
data_rnn = []
for outlier_prob in (0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0):
    print(outlier_prob)
    dataset = Dataset.generate(train_outlier_prob=outlier_prob, n_samples=100)    
    model = get_rnn_model()

    model.fit(
        x=dataset.to_split_window_tfds(window_size=50).batch(32),
        epochs=10,
        verbose=0
    )
    test_dataset = Dataset.generate(train_outlier_prob=outlier_prob, n_samples=20)
    res = model.evaluate(test_dataset.to_split_window_tfds(window_size=50).batch(32), verbose=0)
    data_rnn.append({'outlier_prob': outlier_prob, 'accuracy': res[1]})
    
df_rnn = pd.DataFrame(data_rnn)

0.01
0.05
0.1
0.2
0.3
0.4
0.5
0.6
0.7
0.8
0.9
1.0


In [4]:
alt.Chart(df_rnn).mark_line().encode(x='outlier_prob', y='accuracy')

alt.Chart(...)

Looks quite similar to the per-sample RNN, but the accuracy of at the low levels of noise doesn't reach 0.99 for some reason. I don't know why this is happening, so let me know in the comments if you have an idea.

Let's see again how the tensorboard graphs look like for this RNN:

In [6]:
# Clear any logs from previous runs
from datetime import datetime
!rm -rf ./logs/
log_dir = "logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

dataset = Dataset.generate(train_outlier_prob=0.10, n_samples=200)

get_rnn_model().fit(
    x=dataset.to_split_window_tfds(window_size=50).batch(20),
    epochs=10,
    callbacks=[tensorboard_callback]
)

#%tensorboard --logdir logs/fit

Epoch 1/10
156/156 [==============================] - 5s 23ms/step - loss: 0.3542 - binary_accuracy: 0.8406
Epoch 2/10
156/156 [==============================] - 3s 18ms/step - loss: 0.1942 - binary_accuracy: 0.9414
Epoch 3/10
156/156 [==============================] - 3s 20ms/step - loss: 0.1860 - binary_accuracy: 0.9445
Epoch 4/10
156/156 [==============================] - 3s 19ms/step - loss: 0.1820 - binary_accuracy: 0.9457
Epoch 5/10
156/156 [==============================] - 3s 19ms/step - loss: 0.1795 - binary_accuracy: 0.9464
Epoch 6/10
156/156 [==============================] - 3s 20ms/step - loss: 0.1780 - binary_accuracy: 0.9469
Epoch 7/10
156/156 [==============================] - 3s 20ms/step - loss: 0.1770 - binary_accuracy: 0.9473
Epoch 8/10
156/156 [==============================] - 3s 19ms/step - loss: 0.1764 - binary_accuracy: 0.9475
Epoch 9/10
156/156 [==============================] - 3s 19ms/step - loss: 0.1760 - binary_accuracy: 0.9474
Epoch 10/10
156/156 [=======